# G-Research Crypto Forecasting 

Quick investigation how BTC data in the data set compares to various crypto instruments on Binance.

We compare the randomly selected data points on {sample_date}.


In [ ]:
import pandas as pd

sample_date = "2021-06-01"
train = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/train.csv")
# train = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv")

train = train.query("Asset_ID == 1")
train["DateTime"] = pd.to_datetime(train["timestamp"], unit="s")

train.query(f"DateTime == '{sample_date} 00:01'")


In [ ]:
# coin margined derivatives
url = f"https://data.binance.vision/data/futures/cm/daily/klines/BTCUSD_PERP/1m/BTCUSD_PERP-1m-{sample_date}.zip"
# usd margined derivatives
url = f"https://data.binance.vision/data/futures/um/daily/klines/BTCBUSD/1m/BTCBUSD-1m-{sample_date}.zip"
url = f"https://data.binance.vision/data/futures/um/daily/klines/BTCUSDT/1m/BTCUSDT-1m-{sample_date}.zip"

binance = pd.read_csv(
    url,
    skiprows=1,
    names=["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote asset volume", "Number of trades", "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"]
)
binance["DateTime"] = pd.to_datetime(binance["Open Time"], unit="ms")
# if quote asset volume is in USD, divide by price for BTC volume
binance["BaseVolume"] = binance["Quote asset volume"] / binance["Close"]

binance.query(f"DateTime == '{sample_date} 00:01:00'")


In [ ]:
# spot
url = f"https://data.binance.vision/data/spot/daily/klines/BTCBUSD/1m/BTCBUSD-1m-{sample_date}.zip"
url = f"https://data.binance.vision/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-{sample_date}.zip"

binance_spot = pd.read_csv(
    url,
    skiprows=1,
    names=["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote asset volume", "Number of trades", "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"]
)
binance_spot["DateTime"] = pd.to_datetime(binance_spot["Open Time"], unit="ms")
# if quote asset volume is in USD, divide by price for BTC volume
binance_spot["BaseVolume"] = binance_spot["Quote asset volume"] / binance_spot["Close"]

binance_spot.query(f"DateTime == '{sample_date} 00:01:00'")


In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension("bokeh")
opts.defaults(opts.Curve(tools=["hover"]))

train = train.query("DateTime >= '2021-06-01 00:00' and DateTime < '2021-06-02 00:00'")
hv.Curve((binance_spot.DateTime, binance_spot.BaseVolume), label="Binance Spot") * \
hv.Curve((binance.DateTime, binance.BaseVolume), label="Binance Futures") * \
hv.Curve((train.DateTime, train.Volume), label="Competition Data")


# Conclusion

It looks like the competition data trading volumes are mid way between Binance spot and futures volumes.

There do not appear to be any exact matches with different USD pairs.

My assumption would be that the competition trading volumes are composed of average traded volumes across multiple exchanges as apposed to a cumulative sum.
